# NLP - Assignment 2

## Imports

In [2]:
import pandas as pd
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from bertopic import BERTopic
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer, PorterStemmer
from FuzzyTM import FLSA_W
from gensim.models import CoherenceModel, Phrases
import matplotlib.pyplot as plt
import gensim
from gensim import corpora

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

## Preprocess Data

In [3]:
path = "./us_equities_news_dataset.csv"

# Load the news dataset
news_dataset = pd.read_csv(path)
news_dataset.head()

,id,ticker,title,category,content,release_date,provider,url,article_id
0,221515,NIO,Why Shares of Chinese Electric Car Maker NIO A...,news,What s happening\nShares of Chinese electric c...,2020-01-15,The Motley Fool,https://invst.ly/pigqi,2060327
1,221516,NIO,NIO only consumer gainer Workhorse Group amon...,news,Gainers NIO NYSE NIO 7 \nLosers MGP Ingr...,2020-01-18,Seeking Alpha,https://invst.ly/pje9c,2062196
2,221517,NIO,NIO leads consumer gainers Beyond Meat and Ma...,news,Gainers NIO NYSE NIO 14 Village Farms In...,2020-01-15,Seeking Alpha,https://invst.ly/pifmv,2060249
3,221518,NIO,NIO NVAX among premarket gainers,news,Cemtrex NASDAQ CETX 85 after FY results \n...,2020-01-15,Seeking Alpha,https://invst.ly/picu8,2060039
4,221519,NIO,PLUG NIO among premarket gainers,news,aTyr Pharma NASDAQ LIFE 63 on Kyorin Pharm...,2020-01-06,Seeking Alpha,https://seekingalpha.com/news/3529772-plug-nio...,2053096


In [4]:
# Filter dataset to only include articles with 'Nvidia' in the content
nvidia_dataset = news_dataset[news_dataset['content'].str.contains('Nvidia', case=False, na=False)]

In [5]:
# Concatenate title and content columns
nvidia_dataset['content'] = nvidia_dataset['title'] + ' ' + nvidia_dataset['content']

In [6]:
# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

# Define the default stop words
default_stop_words = set(stopwords.words('english'))

def preprocess_text(text,
                    remove_punctuation=True,
                    remove_stopwords=True,
                    lemmatize=False,
                    remove_short_words=False,
                    remove_numbers=True,
                    min_word_length=2,
                    remove_words=None,  # specific words to remove
                    meaningful_words=None  # meaningful words to keep
                   ):
    """
    Advanced preprocessing function that applies different levels of text processing.

    Parameters:
    - text: The text to preprocess.
    - remove_punctuation: Whether to remove punctuation from the text.
    - remove_stopwords: Whether to remove common stopwords.
    - lemmatize: Whether to apply lemmatization to reduce words to their root form.
    - remove_short_words: Whether to remove short words from the text.
    - remove_numbers: Whether to remove numbers from the text.
    - min_word_length: The minimum length of words to keep in the text.
    - remove_words: A list of specific words to remove.
    - meaningful_words: A list of meaningful words to exclude from stop words.

    Returns:
    - Preprocessed text as tokens.
    """

    # Tokenization
    tokens = word_tokenize(text)

    # Convert to lowercase
    tokens = [token.lower() for token in tokens]

    # Remove numbers if specified
    if remove_numbers:
        tokens = [re.sub(r'\d+', '', token) for token in tokens]

    # Remove non-alphabetic characters (punctuation)
    if remove_punctuation:
        tokens = [re.sub(r'[^a-zA-Z]', '', token) for token in tokens]

    # Remove stopwords, excluding meaningful words
    if remove_stopwords:
        stop_words = default_stop_words
        if meaningful_words:
            stop_words = stop_words - set(meaningful_words)  # Exclude meaningful words from stop words
        tokens = [token for token in tokens if token not in stop_words]

    # Lemmatization
    if lemmatize:
        tokens = [lemmatizer.lemmatize(token) for token in tokens]

    # Remove short words
    if remove_short_words:
        tokens = [token for token in tokens if len(token) >= min_word_length]

    # Calculate global word frequencies
    global_word_frequencies = Counter(tokens)

    # Remove specific words
    if remove_words:
        tokens = [token for token in tokens if token not in remove_words]

    return tokens

In [7]:
def apply_preprocessing(nvidia_dataset, version='v1'):
    """
    Apply different levels of preprocessing to the dataset.

    Parameters:
    - nvidia_dataset: The Nvidia articles with a 'content' column.
    - version: The version of preprocessing to apply ('v1', 'v2', 'v3', or 'v4').

    Returns:
    - DataFrame with the original content and the preprocessed content in 'preprocessed_content' column.
    """

    # Make a copy of the dataset for each version to avoid overwriting the original content
    dataset_copy = nvidia_dataset.copy()

    if version == 'v1':
        # Basic tokenization and lowercasing
        dataset_copy['preprocessed_content'] = dataset_copy['content'].apply(lambda x: preprocess_text(
            x,
            remove_punctuation=False,
            remove_stopwords=False,
            lemmatize=False,
            remove_numbers=False,
            remove_short_words=False
        ))

    elif version == 'v2':
        # Remove punctuation, stopwords, and numbers, but no lemmatization/stemming
        dataset_copy['preprocessed_content'] = dataset_copy['content'].apply(lambda x: preprocess_text(
            x,
            remove_punctuation=True,
            remove_stopwords=True,
            lemmatize=False,
            remove_numbers=True,
            remove_short_words=False,
            remove_words=['nvidia', 'nvda'],
            meaningful_words=['up', 'down']
        ))

    elif version == 'v3':
        # Full preprocessing with lemmatization, number removal, and short words removal
        dataset_copy['preprocessed_content'] = dataset_copy['content'].apply(lambda x: preprocess_text(
            x,
            remove_punctuation=True,
            remove_stopwords=True,
            lemmatize=True,
            remove_numbers=True,
            remove_short_words=True,
            min_word_length=2,
            remove_words=['nvidia', 'nvda'],
            meaningful_words=['up', 'down', 'through', 'during', 'before', 'after', 'above', 'below']
        ))

    else:
        raise ValueError("Invalid preprocessing version. Choose from 'v1', 'v2', 'v3'")

    # Return a separate DataFrame for each version with preprocessed content
    return dataset_copy


In [8]:
train_data_v1 = apply_preprocessing(nvidia_dataset, version='v1')  # Basic preprocessing
train_data_v2 = apply_preprocessing(nvidia_dataset, version='v2')  # Intermediate preprocessing
train_data_v3 = apply_preprocessing(nvidia_dataset, version='v3')  # Full preprocessing with lemmatizer

train_data_v1.head()

,id,ticker,title,category,content,release_date,provider,url,article_id,preprocessed_content
24,221539,NIO,A Central Bank War Just Started And Its Good F...,opinion,A Central Bank War Just Started And Its Good F...,2019-03-07,Michael Kramer,https://www.investing.com/analysis/a-central-b...,200395687,"[a, central, bank, war, just, started, and, it..."
32,221547,NIO,6 Stocks To Watch Nivida Could Be Falling,opinion,6 Stocks To Watch Nivida Could Be Falling 6 S...,2019-03-06,Michael Kramer,https://www.investing.com/analysis/6-stocks-to...,200394931,"[6, stocks, to, watch, nivida, could, be, fall..."
57,221572,NIO,Stocks Dow Drops Nearly 400 Points as Apple ...,news,Stocks Dow Drops Nearly 400 Points as Apple ...,2018-11-19,Investing.com,https://www.investing.com/news/stock-market-ne...,1694042,"[stocks, dow, drops, nearly, 400, points, as, ..."
78,221593,UBER,The Zacks Analyst Blog Highlights Advanced Mi...,opinion,The Zacks Analyst Blog Highlights Advanced Mi...,2020-01-12,Zacks Investment Research,https://www.investing.com/analysis/the-zacks-a...,200498277,"[the, zacks, analyst, blog, highlights, advanc..."
82,221597,UBER,The Best Of CES 2020 Revised,opinion,The Best Of CES 2020 Revised With 4 500 comp...,2020-01-16,Zacks Investment Research,https://www.investing.com/analysis/the-best-of...,200499164,"[the, best, of, ces, 2020, revised, with, 4, 5..."


In [9]:
unique_words_v1 = set([word for sublist in train_data_v1['preprocessed_content'] for word in sublist])
unique_words_v2 = set([word for sublist in train_data_v2['preprocessed_content'] for word in sublist])
unique_words_v3 = set([word for sublist in train_data_v3['preprocessed_content'] for word in sublist])

print("Unique words in v1:", len(unique_words_v1))
print("Unique words in v2:", len(unique_words_v2))
print("Unique words in v3:", len(unique_words_v3))

Unique words in v1: 36322
Unique words in v2: 32137
Unique words in v3: 29056


In [10]:
# Flatten lists and count word frequencies
freq_v1 = Counter([word for sublist in train_data_v1['preprocessed_content'] for word in sublist])
freq_v2 = Counter([word for sublist in train_data_v2['preprocessed_content'] for word in sublist])
freq_v3 = Counter([word for sublist in train_data_v3['preprocessed_content'] for word in sublist])

# Print the most common words side by side for easier comparison
def print_side_by_side(freq_v1, freq_v2, freq_v3, top_n=10):
    print(f"{'Rank':<5} {'v1':<20} {'Count':<10} {'v2':<20} {'Count':<10} {'v3':<20} {'Count':<10}")
    print("="*100)
    for i, ((word1, count1), (word2, count2), (word3, count3)) in enumerate(zip(freq_v1.most_common(top_n), freq_v2.most_common(top_n), freq_v3.most_common(top_n)), 1):
        print(f"{i:<5} {word1:<20} {count1:<10} {word2:<20} {count2:<10} {word3:<20} {count3:<10}")

print_side_by_side(freq_v1, freq_v2, freq_v3)

Rank  v1                   Count      v2                   Count      v3                   Count     
1     the                  126904                          144555     stock                16293     
2     to                   61701      nasdaq               14831      year                 16074     
3     and                  55488      year                 13876      company              15430     
4     of                   54474      zacks                11672      nasdaq               14831     
5     in                   46306      company              11343      zacks                11672     
6     a                    44793      earnings             10457      market               10686     
7     s                    32960      stocks               8935       earnings             10457     
8     is                   26139      market               8385       quarter              8718      
9     for                  24701      quarter              7809       share       

In [11]:
# Flatten the list of lists to get a single list of words
all_tokens_v3 = [word for sublist in train_data_v3['preprocessed_content'] for word in sublist]

# Get the 50 most common words
common_words_v3 = Counter(all_tokens_v3).most_common(50)

# Print the most common 50 words and their respective frequencies
print('Most common 50 words and respective frequencies:\n')
for i, (word, freq) in enumerate(common_words_v3, 1):
    print(f'{i:<2}- {word:<15} {freq}')

Most common 50 words and respective frequencies:

1 - stock           16293
2 - year            16074
3 - company         15430
4 - nasdaq          14831
5 - zacks           11672
6 - market          10686
7 - earnings        10457
8 - quarter         8718
9 - share           8282
10- growth          7409
11- revenue         7334
12- up              6939
13- billion         6322
14- also            6220
15- nyse            6159
16- technology      5586
17- estimate        5547
18- new             5150
19- investor        5036
20- million         4806
21- rank            4789
22- last            4759
23- week            4594
24- inc             4564
25- expected        4280
26- after           4269
27- day             4199
28- report          4161
29- price           4134
30- buy             4067
31- trade           4037
32- investment      3904
33- one             3874
34- time            3862
35- data            3856
36- high            3855
37- strong          3688
38- per           

In [12]:
print(f"Number of unique words: {len(set(all_tokens_v3)):,}")
print(f"Lexical richness: {len(set(all_tokens_v3))/len(all_tokens_v3):.4f}")

Number of unique words: 29,056
Lexical richness: 0.0210


## Helper functions

In [13]:
def remove_top_bottom_words(preprocessed_content, top_percent=10, bottom_percent=10, words_to_exclude=None):
    """
    Remove top-n% and bottom-n% words based on frequency and exclude specified words.

    Parameters:
    - preprocessed_content: list of lists, the preprocessed content
    - top_percent: int, the top n% of words to remove
    - bottom_percent: int, the bottom n% of words to remove
    - words_to_exclude: set, the words to exclude

    Returns:
    - list of lists, the content with specified words removed
    """
    # Flatten the list of lists to get a single list of words
    all_words = [word for sublist in preprocessed_content for word in sublist]

    # Get the word frequencies
    word_freq = Counter(all_words)

    # Calculate the number of words to remove
    total_words = len(word_freq)
    top_n = int(total_words * top_percent / 100)
    bottom_n = int(total_words * bottom_percent / 100)

    # Get the top-n% and bottom-n% words
    top_words = set([word for word, freq in word_freq.most_common(top_n)])
    bottom_words = set([word for word, freq in word_freq.most_common()[:-bottom_n-1:-1]])

    # Combine the words to exclude
    words_to_remove = top_words | bottom_words
    if words_to_exclude:
        words_to_remove |= words_to_exclude

    # Remove the specified words from the content
    filtered_content = [[word for word in sublist if word not in words_to_remove] for sublist in preprocessed_content]

    return filtered_content

In [14]:
def print_most_common_bigrams(bigrams, train_data_list_tokens, top_n=20):
    """
    Print the most common bigrams in the tokenized text data.

    Parameters:
    - bigrams: the bigrams created (if any)
    - train_data_list_tokens: list of list of str, the tokenized text data
    - top_n: int, the number of most common bigrams to print
    """
    if bigrams is None:
        print("No bigrams were created.")
        return

    # Transform the tokenized text data to include bigrams
    bigram_mod = bigrams[train_data_list_tokens]

    # Flatten the list of lists to get a single list of tokens
    all_tokens = [token for doc in bigram_mod for token in doc]

    # Count the frequency of each token
    token_freq = Counter(all_tokens)

    # Extract and count the bigrams
    bigram_freq = Counter({token: freq for token, freq in token_freq.items() if '_' in token})

    # Print the most common bigrams
    print(f'Top {top_n} most common bigrams and respective frequencies:\n')
    for i, (bigram, freq) in enumerate(bigram_freq.most_common(top_n), 1):
        print(f'{i:<2}- {bigram:<30} {freq}')

### LDA

In [15]:
def lda_print_evaluation_metrics(lda_model, train_data_list, dictionary):
    """
    Print the evaluation metrics for a trained LDA model, including coherence score.

    Parameters:
    - lda_model: Trained LDA model.
    - train_data_list: List of tokenized documents (for coherence calculation).
    - dictionary: Gensim dictionary.
    """
    # Show topics with word probabilities
    print("\n\nLDA Topics with Word Probabilities:\n")
    for idx, topic in lda_model.print_topics():
        print(f"Topic {idx + 1}: {topic}")

    # Vocabulary size
    print(f"\nVocabulary size: {len(dictionary):,}")

    # Coherence Score Calculation
    coherence_model = CoherenceModel(
        model=lda_model,
        texts=train_data_list,
        dictionary=dictionary,
        coherence='c_v'
    )
    coherence_score = coherence_model.get_coherence()
    print("\nCoherence Score:", round(coherence_score, 4))


### FLSA-W

In [16]:
def compute_coherence_flsa_w(train_data, max_topics=10, num_words_list=[5, 7, 10]):
    """
    Compute coherence scores for FLSA-W model with varying number of topics and words per topic.

    Parameters:
        - train_data: list of tokenized documents (preprocessed content).
        - max_topics: int, the maximum number of topics to test.
        - num_words_list: list of int, different numbers of words per topic to test.
        - display: bool, if True, will print the results in a structured format per num_words.

    Returns:
        - results: list of tuples, each containing (num_words, num_topics, coherence_score).
    """

    tokenized_data = [tokens for tokens in train_data['preprocessed_content']]

    results = []

    # Loop through each number of words per topic first
    for num_words in num_words_list:
        print(f"\n=== Results for {num_words} words per topic ===")

        # Loop through each number of topics for the current number of words
        for num_topics in range(2, max_topics + 1):
            # Train FLSA-W model with the current num_words and num_topics
            flsa_w_model = FLSA_W(
                input_file=tokenized_data,
                num_topics=num_topics,
                num_words=num_words
            )

            # Train the FLSA-W model
            flsa_w_model.get_matrices()

            # Get the coherence score for the current model
            coherence_score = flsa_w_model.get_coherence_score()

            # Store the results as a tuple (num_words, num_topics, coherence_score)
            results.append((num_words, num_topics, coherence_score))

            print(f"Number of topics: {num_topics}, Coherence score: {coherence_score:.4f}")

    return results


In [17]:
def plot_elbow_curve(results):
    """
    Create elbow plots for coherence scores vs. number of topics for different num_words.

    Parameters:
    - results: list of tuples, each containing (num_words, num_topics, coherence_score).
    """
    # Organize results by num_words
    num_words_dict = {}
    for num_words, num_topics, coherence_score in results:  # Corrected tuple unpacking order
        if num_words not in num_words_dict:
            num_words_dict[num_words] = {'num_topics': [], 'coherence_scores': []}
        num_words_dict[num_words]['num_topics'].append(num_topics)
        num_words_dict[num_words]['coherence_scores'].append(coherence_score)

    # Plot each num_words line
    plt.figure(figsize=(10, 6))
    for num_words, data in num_words_dict.items():
        plt.plot(data['num_topics'], data['coherence_scores'], marker='o', label=f'Num words: {num_words}')

    # Add plot labels and title
    plt.title('FLSA-W Elbow Plot: Number of Topics vs Coherence Score for Different Number of Words per Topic')
    plt.xlabel('Number of Topics')
    plt.ylabel('Coherence Score')
    plt.legend(title='Words per Topic')
    plt.grid(True)
    plt.show()

In [18]:
def flsa_w_print_evaluation_metrics(flsa_w_model, flsa_w_topics):
    """
    Print the evaluation metrics for the trained FLSA-W model.

    Parameters:
    - flsa_w_model: the trained FLSA-W model
    - flsa_w_topics: the topics generated by the FLSA-W model
    """
    # Show the topics generated with word probabilities
    print("\n\nFLSA-W Topics with Word Probabilities:\n")
    for topic in flsa_w_model.show_topics():
        print(topic)

    # Show the topics generated
    print("\n\nFLSA-W Topics:\n")
    for i, topic in enumerate(flsa_w_topics):
        print(f"Topic {i + 1}: {', '.join(topic)}")

    print(f"\nVocabulary size: {flsa_w_model.get_vocabulary_size():,}")

    # Show topic quality scores
    print("\nCoherence Score:", round(flsa_w_model.get_coherence_score(), 4))
    print("Diversity Score:", round(flsa_w_model.get_diversity_score(), 4))
    print("Interpretability Score:", round(flsa_w_model.get_interpretability_score(), 4))

### BERTopic

In [40]:
def bertopic_print_evaluation_metrics(topic_model, topics, train_data_token_list, dictionary):
    """
    Print evaluation metrics of a trained BERTopic model.

    Parameters:
    - topic_model: the trained BERTopic model.
    - topics: the topics generated by the BERTopic model.
    - train_data_token_list: list of tokenized documents (preprocessed content).
    - dictionary: Gensim dictionary.
    """

    print("BERTopic Topic Info:\n")
    print(topic_model.get_topic_info())

    # Show the topics generated
    print("\nBERTopic Topics with Word Probabilities:\n")
    for topic_id in topics:  # Iterate through topic IDs
        topic_words = topic_model.get_topic(topic_id)  # Get words for the topic
        print(f"Topic {topic_id}: {topic_words}") # Print words and probabilities


    # Extract topics as lists of tokens (words)
    topic_tokens = [[word for word, _ in topic_model.get_topic(topic_id)]
                        for topic_id in topics.keys() if topic_id != -1]  # Exclude outlier topic -1

    # Compute coherence score
    coherence_model = CoherenceModel(topics=topic_tokens,  # Pass list of token lists
                                     texts=train_data_token_list,
                                     dictionary=dictionary,
                                     coherence='c_v')

    coherence_score = coherence_model.get_coherence()

    print(f"\nCoherence Score: {coherence_score:.4f}")

## Elbow Plot

In [ ]:
# Set seed for reproducible results
np.random.seed(42)

# Train the model and compute coherence scores
results = compute_coherence_flsa_w(train_data_v3, max_topics=15, num_words_list=[5, 7, 10]) # FYI only did this for v3

In [ ]:
# Plot the elbow curve
plot_elbow_curve(results)

## Train Topic Models

In [20]:
# the optimal number of topics according to FLSA-W
NUMBER_OF_TOPICS = 5

# Dataset has low lexical richness, so we use a smaller number of words per topic
NUMBER_OF_WORDS_PER_TOPIC = 5

In [21]:
def train_topic_model(train_data, model_type='LDA', num_topics=NUMBER_OF_TOPICS, num_words_per_topic=NUMBER_OF_WORDS_PER_TOPIC, preprocess=False, top_percent=0, bottom_percent=0, words_to_exclude=None, use_bigrams=False):
    """
    Train a topic model on the given training data.

    Parameters:
    - train_data: dict, the text to train the model on with key 'preprocessed_content'
    - model_type: str, the type of model to train ('LDA', 'FLSA-W', 'BERTopic')
    - num_topics: int, the number of topics to use for the model
    - num_words_per_topic: int, the number of words per topic to use for the model
    - preprocess: bool, whether to preprocess the data using remove_top_bottom_words
    - top_percent: int, the top n% of words to remove based on frequency
    - bottom_percent: int, the bottom n% of words to remove based on frequency
    - words_to_exclude: list of str, specific words to exclude from the data
    - use_bigrams: bool, whether to include bigrams in the data

    Returns:
    - model: the trained model
    - topics: the topics generated by the model
    - bigrams: the bigrams created (if any)
    """
    # Set seed for reproducible results
    np.random.seed(42)

    train_data_list_tokens = train_data['preprocessed_content']
    bigrams = None

    if preprocess:
        train_data_list_tokens = remove_top_bottom_words(
            train_data_list_tokens,
            top_percent=top_percent,
            bottom_percent=bottom_percent,
            words_to_exclude=words_to_exclude
        )

    if use_bigrams:
        # Detect bigrams in the tokenized text data
        bigram = Phrases(train_data_list_tokens,
                         min_count=5, # min number of times a bigram must appear in the corpus to be considered a phrase
                         threshold=90) # scoring threshold for forming phrases based on the likelihood of the words appearing together versus separately.
        train_data_list_tokens = [bigram[doc] for doc in train_data_list_tokens]
        bigrams = bigram

    # Filter out empty lists
    train_data_list_tokens = [tokens for tokens in train_data_list_tokens if tokens]
    train_data_list_string = [' '.join(tokens) for tokens in train_data_list_tokens]

    if model_type == 'LDA':
        # Create Gensim dictionary
        dictionary = corpora.Dictionary(train_data_list_tokens)

        # Remove very common and very rare words from dictionary
        dictionary.filter_extremes(no_below=0.5, no_above=0.5)

        # Create Document-Term Matrix
        doc_term_matrix = [dictionary.doc2bow(doc) for doc in train_data_list_tokens]

        lda_model = gensim.models.LdaModel(
            doc_term_matrix,
            num_topics=num_topics,
            id2word=dictionary,
            random_state=42,
            passes=5,
            alpha='auto'
        )

        # Get topics (top words in each topic)
        topics = lda_model.print_topics(num_words=num_words_per_topic)

        return lda_model, topics, bigrams, train_data_list_tokens, dictionary

    elif model_type == 'FLSA-W':
        flsa_w_model = FLSA_W(
            input_file=train_data_list_tokens,
            num_topics=num_topics,
            num_words=num_words_per_topic
        )
        pwgt, ptgd = flsa_w_model.get_matrices()  # This trains the model
        topics = flsa_w_model.show_topics(representation='words')

        return flsa_w_model, topics, bigrams, train_data_list_tokens, None

    elif model_type == 'BERTopic':
        # Create Gensim dictionary
        dictionary = corpora.Dictionary(train_data_list_tokens)

        # Train BERTopic model
        topic_model = BERTopic(nr_topics = num_topics + 1, top_n_words = num_words_per_topic) # num_topics + 1 to disregard the noise (-1) topic
        _, _ = topic_model.fit_transform(train_data_list_string)

        topics = topic_model.get_topics()

        return topic_model, topics, bigrams, train_data_list_tokens, dictionary

    else:
        raise ValueError("Invalid model_type. Choose from 'LDA', 'FLSA-W', 'BERTopic'.")

## Iteration 1

### LDA

In [ ]:
LDA_model, LDA_topics, bigrams, train_data_list_tokens, dictionary = train_topic_model(train_data_v1, model_type='LDA')

print_most_common_bigrams(bigrams, train_data_list_tokens, top_n=15)

# Set up topics in the correct format for CoherenceModel
lda_topics = [[word for word, _ in LDA_model.show_topic(topic_id, topn=10)] for topic_id in range(LDA_model.num_topics)]

# Call the modified evaluation function
lda_print_evaluation_metrics(LDA_model, train_data_list_tokens, dictionary)

### FLSA-W

In [ ]:
# Train FLSA-W model on the first version of the preprocessed dataset
flsa_w_model, flsa_w_topics, bigrams, train_data_list_tokens, _ = train_topic_model(train_data_v1,
                                                         model_type='FLSA-W')

# Print the most common bigrams
print_most_common_bigrams(bigrams, train_data_list_tokens, top_n=15)

# Print evaluation metrics for the FLSA-W model
flsa_w_print_evaluation_metrics(flsa_w_model, flsa_w_topics)

### BERTopic

In [23]:
# Train BERTopic model on the first version of the preprocessed dataset
bertopic_model, bertopic_topics, bigrams, train_data_list_tokens, dictionary = train_topic_model(train_data_v1, model_type='BERTopic')

# Print the most common bigrams
print_most_common_bigrams(bigrams, train_data_list_tokens, top_n=15)

# Evaluation metrics of the BERTopic model
bertopic_print_evaluation_metrics(bertopic_model, bertopic_topics, train_data_list_tokens, dictionary)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

No bigrams were created.
BERTopic Topic Info:

   Topic  Count                    Name                 Representation  \
0     -1   1178        -1_the_of_and_to         [the, of, and, to, in]   
1      0   1839         0_the_to_and_of         [the, to, and, of, in]   
2      1    175    1_the_to_and_driving    [the, to, and, driving, in]   
3      2    149  2_traded_nvidia_at_the  [traded, nvidia, at, the, on]   
4      3     34   3_facebook_the_to_and   [facebook, the, to, and, in]   
5      4     25   4_the_to_qualcomm_and   [the, to, qualcomm, and, in]   

                                 Representative_Docs  
0  [tech roundup nvda pcln trip earnings intc amd...  
1  [the zacks analyst blog highlights nvidia adva...  
2  [can intel challenge nvidia in the self drivin...  
3  [nvidia rises 4 investing com nvidia nasdaq nv...  
4  [facebook fb launches tools to prevent misuse ...  
5  [tech stock roundup aapl tax bill googl nest t...  

BERTopic Topics with Word Probabilities:

Topic 

## Iteration 2

### LDA

In [ ]:
LDA_model, LDA_topics, bigrams, train_data_list_tokens, dictionary = train_topic_model(
    train_data_v2,
    model_type='LDA',
    preprocess=True,
    top_percent=2,
    bottom_percent=2,
    #words_to_exclude={'u', 'p', 'amd', 'inc', 'etf', 'q'} TODO Higher coherence without removing these...
    )

print_most_common_bigrams(bigrams, train_data_list_tokens, top_n=15)

# Set up topics in the correct format for CoherenceModel
lda_topics = [[word for word, _ in LDA_model.show_topic(topic_id, topn=10)] for topic_id in range(LDA_model.num_topics)]

# Call the modified evaluation function
lda_print_evaluation_metrics(LDA_model, train_data_list_tokens, dictionary)

### FLSA-W

In [ ]:
# Train FLSA-W model on the first version of the preprocessed dataset
flsa_w_model, flsa_w_topics, bigrams, train_data_list_tokens, _ = train_topic_model(train_data_v2,
                                                model_type='FLSA-W',
                                                preprocess=True,
                                                words_to_exclude={'gmt', 'inc', 'com', 'whileolympusshares', 'upnoteus'})

# Print the most common bigrams
print_most_common_bigrams(bigrams, train_data_list_tokens, top_n=15)

# Print evaluation metrics for the FLSA-W model
flsa_w_print_evaluation_metrics(flsa_w_model, flsa_w_topics)

### BERTopic

In [24]:
# Train BERTopic model on the second version of the preprocessed dataset
bertopic_model, bertopic_topics, bigrams, train_data_list_tokens, dictionary = train_topic_model(train_data_v2, model_type='BERTopic')

# Print the most common bigrams
print_most_common_bigrams(bigrams, train_data_list_tokens, top_n=15)

# Evaluation metrics of the BERTopic model
bertopic_print_evaluation_metrics(bertopic_model, bertopic_topics, train_data_list_tokens, dictionary)

No bigrams were created.
BERTopic Topic Info:

   Topic  Count                             Name  \
0     -1   1085      -1_year_nasdaq_zacks_stocks   
1      0   2097      0_nasdaq_year_zacks_company   
2      1    131        1_traded_seven_lowest_gmt   
3      2     51        2_gaming_games_game_video   
4      3     21    3_qualcomm_commission_eu_said   
5      4     15  4_markets_italy_rallied_climbed   

                                Representation  \
0       [year, nasdaq, zacks, stocks, company]   
1     [nasdaq, year, zacks, company, earnings]   
2        [traded, seven, lowest, gmt, highest]   
3         [gaming, games, game, video, nasdaq]   
4  [qualcomm, commission, eu, said, antitrust]   
5   [markets, italy, rallied, climbed, soared]   

                                 Representative_Docs  
0  [potlatch signet jewelers zacks bull bear day ...  
1  [mastec orchid island applied materials nxp am...  
2  [falls  investing com nasdaq fell   trade     ...  
3  [zacks analyst

## Iteration 3

### LDA

In [ ]:
# TODO: lower coherence then iteration 2....
LDA_model, LDA_topics, bigrams, train_data_list_tokens, dictionary = train_topic_model(
    train_data_v3,
    model_type='LDA',
    preprocess=True,
    words_to_exclude={'amd', 'inc', 'etf', 'sma', 'et', 'gmt', 'dma', 'baidu', 'radeon', 'vix'},
    top_percent=2,
    bottom_percent=2
    )

print_most_common_bigrams(bigrams, train_data_list_tokens, top_n=15)

# Call the modified evaluation function
lda_print_evaluation_metrics(LDA_model, train_data_list_tokens, dictionary)

### FLSA-W

In [ ]:
# Train FLSA-W model on the first version of the preprocessed dataset
flsa_w_model, flsa_w_topics, bigrams, train_data_list_tokens, _ = train_topic_model(train_data_v3,
                                                model_type='FLSA-W',
                                                preprocess=True,
                                                words_to_exclude={'gmt', 'inc', 'com', 'whileolympusshares', 'upnoteus', 'googl', 'amat', 'et'})

# Print the most common bigrams
print_most_common_bigrams(bigrams, train_data_list_tokens, top_n=15)

# Print evaluation metrics for the FLSA-W model
flsa_w_print_evaluation_metrics(flsa_w_model, flsa_w_topics)

### BERTopic

In [24]:
# Train BERTopic model on the second version of the preprocessed dataset
bertopic_model, bertopic_topics, bigrams, train_data_list_tokens, dictionary = train_topic_model(train_data_v2,
                                                                                                 model_type='BERTopic',
                                                                                                 preprocess=True,
                                                                                                 top_percent=5,
                                                                                                 bottom_percent=5)

# Print the most common bigrams
print_most_common_bigrams(bigrams, train_data_list_tokens, top_n=15)

# Evaluation metrics of the BERTopic model
bertopic_print_evaluation_metrics(bertopic_model, bertopic_topics, train_data_list_tokens, dictionary)

No bigrams were created.
BERTopic Topic Info:

   Topic  Count                                           Name  \
0     -1   1250                     -1_ansys_beta_mutual_stake   
1      0   1919        0_automakers_tracing_ethernet_extension   
2      1     76                                     1_rises___   
3      2     53     2_commission_hikvision_surveillance_facial   
4      3     16  3_italy_australian_italian_whileolympusshares   
5      4     13                 4_microchip_mchp_sam_microsemi   

                                      Representation  \
0                 [ansys, beta, mutual, stake, macy]   
1   [automakers, tracing, ethernet, extension, epyc]   
2                                    [rises, , , , ]   
3  [commission, hikvision, surveillance, facial, ...   
4  [italy, australian, italian, whileolympusshare...   
5           [microchip, mchp, sam, microsemi, atmel]   

                                 Representative_Docs  
0  [roundup probe newsfeed senators callin

## Iteration 4

### LDA

In [ ]:
LDA_model, LDA_topics, bigrams, train_data_list_tokens, dictionary = train_topic_model(
    train_data_v3,
    model_type='LDA',
    preprocess=True,
    words_to_exclude={'amd', 'inc', 'etf', 'sma', 'et', 'gmt', 'dma', 'baidu', 'radeon', 'vix'},
    top_percent=2,
    bottom_percent=2,
    use_bigrams=True
    )

print_most_common_bigrams(bigrams, train_data_list_tokens, top_n=15)

# Call the modified evaluation function
lda_print_evaluation_metrics(LDA_model, train_data_list_tokens, dictionary)

### FLSA-W

In [ ]:
# Train FLSA-W model on the first version of the preprocessed dataset
flsa_w_model, flsa_w_topics, bigrams, train_data_list_tokens, _ = train_topic_model(train_data_v3,
                                                model_type='FLSA-W',
                                                preprocess=True,
                                                words_to_exclude={'gmt', 'inc', 'com', 'whileolympusshares', 'upnoteus', 'googl', 'amat', 'et'},
                                                use_bigrams=True)

# Print the most common bigrams
print_most_common_bigrams(bigrams, train_data_list_tokens, top_n=15)

# Print evaluation metrics for the FLSA-W model
flsa_w_print_evaluation_metrics(flsa_w_model, flsa_w_topics)

### BERTopic

In [29]:
# Train BERTopic model on the third version of the preprocessed dataset
bertopic_model, bertopic_topics, bigrams, train_data_list_tokens, dictionary = train_topic_model(train_data_v3,
                                                                                                 model_type='BERTopic',
                                                                                                 preprocess=True,
                                                                                                 top_percent=5,
                                                                                                 bottom_percent=5)

# Print the most common bigrams
print_most_common_bigrams(bigrams, train_data_list_tokens, top_n=15)

# Evaluation metrics of the BERTopic model
bertopic_print_evaluation_metrics(bertopic_model, bertopic_topics, train_data_list_tokens, dictionary)

No bigrams were created.
BERTopic Topic Info:

   Topic  Count                                           Name  \
0     -1   1553                        -1_ansys_item_beta_opec   
1      0   1609                      0_ethernet_tracing_rsi_rx   
2      1     32                      1_peg_dinrf_premarket_stm   
3      2     28                  2_rbc_crest_lipacis_jefferies   
4      3     16  3_italy_australian_italian_whileolympusshares   
5      4     13                 4_microchip_mchp_sam_microsemi   

                                      Representation  \
0                [ansys, item, beta, opec, monetary]   
1          [ethernet, tracing, rsi, rx, coronavirus]   
2              [peg, dinrf, premarket, stm, audited]   
3          [rbc, crest, lipacis, jefferies, pacific]   
4  [italy, australian, italian, whileolympusshare...   
5           [microchip, mchp, sam, microsemi, atmel]   

                                 Representative_Docs  
0  [roundup wdc toshiba row surface nok wd

## Iteration 5

### BERTopic

In [32]:
# Train BERTopic model on the third version of the preprocessed dataset
bertopic_model, bertopic_topics, bigrams, train_data_list_tokens, dictionary = train_topic_model(train_data_v2,
                                                                                                 model_type='BERTopic',
                                                                                                 preprocess=True,
                                                                                                 top_percent=5,
                                                                                                 bottom_percent=5,
                                                                                                 words_to_exclude = {'', 'whileolympusshares', 'upnoteus', 'sam'}) # TODO: '' is still in topics

# Print the most common bigrams
print_most_common_bigrams(bigrams, train_data_list_tokens, top_n=15)

# Evaluation metrics of the BERTopic model
bertopic_print_evaluation_metrics(bertopic_model, bertopic_topics, train_data_list_tokens, dictionary)

No bigrams were created.
BERTopic Topic Info:

   Topic  Count                                        Name  \
0     -1   1357                  -1_ansys_beta_mutual_stake   
1      0   1810     0_tracing_ethernet_automakers_extension   
2      1     76                                  1_rises___   
3      2     55  2_commission_hikvision_surveillance_facial   
4      3     16    3_italy_australian_italian_equitiesasian   
5      4     13            4_microchip_mchp_microsemi_atmel   

                                      Representation  \
0            [ansys, beta, mutual, stake, premarket]   
1   [tracing, ethernet, automakers, extension, epyc]   
2                                    [rises, , , , ]   
3  [commission, hikvision, surveillance, facial, ...   
4  [italy, australian, italian, equitiesasian, su...   
5       [microchip, mchp, microsemi, atmel, dspicch]   

                                 Representative_Docs  
0  [roundup flaw avere moneygram flaws avere purc...  
1  [afte

## Iteration 5

### BERTopic

In [41]:
# Train BERTopic model on the third version of the preprocessed dataset
bertopic_model, bertopic_topics, bigrams, train_data_list_tokens, dictionary = train_topic_model(train_data_v2,
                                                                                                 model_type='BERTopic',
                                                                                                 preprocess=True,
                                                                                                 top_percent=5,
                                                                                                 bottom_percent=5,
                                                                                                 words_to_exclude = {'', 'whileolympusshares', 'upnoteus', 'sam'},
                                                                                                 use_bigrams=True) # TODO: fix error when using bigrams

# Print the most common bigrams
print_most_common_bigrams(bigrams, train_data_list_tokens, top_n=15)

# Evaluation metrics of the BERTopic model
bertopic_print_evaluation_metrics(bertopic_model, bertopic_topics, train_data_list_tokens, dictionary)

{-1: [('beta', 0.0056015423187012366), ('mutual', 0.004100772444398326), ('leadership', 0.0038869220921301053), ('continuing', 0.0038486830787187137), ('israel', 0.0038002498578135094)], 0: [('broke', 0.0040867819746371135), ('automakers', 0.003955771406391944), ('monetary', 0.0038600672594631064), ('macy', 0.0036998651084432285), ('unemployment', 0.0036982309493470015)], 1: [('rises', 6.04380033070167), ('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05)], 2: [('commission', 0.07193193043224015), ('hikvision', 0.0648556451464062), ('surveillance', 0.06107777982913537), ('patents', 0.04977535390345285), ('court', 0.044741323056109415)], 3: [('senatesteps', 0.24868671420592053), ('stagedjumping', 0.24868671420592053), ('asxhang', 0.24868671420592053), ('tickedplaced', 0.24868671420592053), ('mibitalian', 0.24868671420592053)], 4: [('compares', 0.19986131722928077), ('outsideaudited', 0.1805730616674073), ('pegpeg', 0.16801521846464063), ('actionable', 0.16511251389809634), ('groups', 0.1

ValueError: unable to interpret topic as either a list of tokens or a list of ids

## Output of Final Topic Model

### LDA

In [ ]:
# Iteration 2:

LDA_model, LDA_topics, bigrams, train_data_list_tokens, dictionary = train_topic_model(
    train_data_v2,
    model_type='LDA',
    preprocess=True,
    top_percent=2,
    bottom_percent=2,
    #words_to_exclude={'u', 'p', 'amd', 'inc', 'etf', 'q'} TODO Higher coherence without removing these...
    )

print_most_common_bigrams(bigrams, train_data_list_tokens, top_n=15)

# Set up topics in the correct format for CoherenceModel
lda_topics = [[word for word, _ in LDA_model.show_topic(topic_id, topn=10)] for topic_id in range(LDA_model.num_topics)]

# Call the modified evaluation function
lda_print_evaluation_metrics(LDA_model, train_data_list_tokens, dictionary)

### FLSA-W

In [ ]:
# Print evaluation metrics for the FLSA-W model
flsa_w_print_evaluation_metrics(flsa_w_model, flsa_w_topics)

### BERTopic

In [ ]:
# Evaluation metrics of the final BERTopic model
bertopic_print_evaluation_metrics(bertopic_model, bertopic_topics, train_data_list_tokens, dictionary)

Topic Similarity Matrix

In [ ]:
bertopic_model.visualize_heatmap()